<a href="https://colab.research.google.com/github/BannyM/Kaggle_Titanic_Competition/blob/master/Machine_Learning_NB_Model_for_Kaggle_Titanic_Survivor_Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import string
import io

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [3]:
df = pd.read_csv(io.StringIO(uploaded['train.csv'].decode('utf-8')))
test = pd.read_csv(io.StringIO(uploaded['test.csv'].decode('utf-8')))
test_ids = test['PassengerId']


In [4]:
#dropping unnecesary columns
unusable_cols = ['PassengerId','Name','SibSp','Parch','Ticket','Fare','Cabin']
df.drop(unusable_cols,axis=1,inplace=True)

df.Age.fillna(df['Age'].mean(),inplace=True) #Replace null values in the Age column with the mean age
df.dropna(subset=['Embarked'],inplace=True) #Drop the two rows that contain null values in Embarked

#Transforming Sex column to numerical category.  Male = 1 and Female = 0
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex']) 

#Create dummy variables for Embarked column
embark_dummies = pd.get_dummies(df.Embarked)
df2 = pd.concat([df,embark_dummies],axis=1)
df2.drop(['Embarked','S'],axis=1,inplace=True)

df2.head()

,Survived,Pclass,Sex,Age,C,Q
0,0,3,1,22.0,0,0
1,1,1,0,38.0,1,0
2,1,3,0,26.0,0,0
3,1,1,0,35.0,0,0
4,0,3,1,35.0,0,0


In [5]:
#Data Cleaning Process for the test set so that it is formatting exactly like the training set [418 rows x 4 columns]>
test.drop(unusable_cols,axis=1,inplace=True) #drop unnecesary columns
test.Age.fillna(test['Age'].mean(),inplace=True)  #Replace null values in the Age column with the mean age

#Transforming Sex column to numerical category.  Male = 1 and Female = 0
test['Sex'] = le.fit_transform(test['Sex']) 

#Create dummy variables for Embarked column
embark_test_dummies = pd.get_dummies(test.Embarked)
test_df = pd.concat([test,embark_test_dummies],axis=1)

test_df.drop(['Embarked','S'],axis=1,inplace=True)

test_df.head()

,Pclass,Sex,Age,C,Q
0,3,1,34.5,0,1
1,3,0,47.0,0,0
2,2,1,62.0,0,1
3,3,1,27.0,0,0
4,3,0,22.0,0,0


In [6]:
#Training the ML Model
y = df2['Survived']
X = df2.drop('Survived',axis=1)

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.20,random_state=11) #Split the training and testing set to 80% and 20%, respectively. 

clf = LogisticRegression(random_state=0,max_iter=1000).fit(X_train,y_train)
predictions = clf.predict(X_val)
accuracy_score(y_val,predictions)

0.8539325842696629

In [7]:
#Creating a csv file for the test.csv to submit to Kaggle 
test_predictions = clf.predict(test_df)

df3 = pd.DataFrame({'PassengerId':test_ids.values,
                    'Survived': test_predictions})          

df3.to_csv('titanic_submission.csv',index=False) 
files.download('titanic_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>